# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [20]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from collections import defaultdict

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
        
def cities_nhl_teams_mapping(df, cities):
    teams_2018 = df[df['year'] == 2018]
    team_names = teams_2018['team']
    team_city_match_found = 0
    team_city_no_match = 0

    # Map of cleaned city name to original city name
    cleaned_cities = {city.split('City')[0].split('Area')[0].split(', D.C.')[0].strip():city for city in cities}
    #print("No. of cities:{} and cleaned_cities:{}".format(len(nhl_cities), len(cleaned_cities)))
    print(f"cities_teams_mapping :: Cleaned Cities {cleaned_cities}")
    
    # Map of original city name to list of teams
    city_teams = defaultdict(list)
    city_teams['Phoenix'].append('Arizona Coyotes')
    city_teams['Las Vegas'].append('Vegas Golden Knights*')
    city_teams['Miami–Fort Lauderdale'].append('Florida Panthers')
    city_teams['Raleigh'].append('Carolina Hurricanes')
    city_teams['Denver'].append('Colorado Avalanche*')
    city_teams['Dallas–Fort Worth'].append('Dallas Stars')
    city_teams['San Francisco Bay Area'].append('San Jose Sharks*')
    city_teams['Minneapolis–Saint Paul'].append('Minnesota Wild*')
    city_teams['Los Angeles'].append('Anaheim Ducks*')
    city_teams['New York City'].append('New Jersey Devils*')

    match_already_found = [team for team_list in city_teams.values() for team in team_list]
    for team in team_names:
        #team = team.split('*')[0] # Some teams have '*' in the last
        if team in match_already_found:
            team_city_match_found += 1
        else:
            back_city_name = team.rsplit(' ', 1)[0].strip() # "New York Islanders" --> "New York"
            front_city_name = team.split(' ')[0].strip() # Toronto Maple Leafs --> Toronto
            if back_city_name in cleaned_cities:
                city_teams[cleaned_cities[back_city_name]].append(team)
                team_city_match_found += 1
            elif front_city_name in cleaned_cities:
                city_teams[cleaned_cities[front_city_name]].append(team)
                team_city_match_found += 1
            else:
                print(f"No city found for {team}")
                team_city_no_match += 1
    #print(f"cities_teams_mapping :: team_city_match_found={team_city_match_found}, team_city_no_match={team_city_no_match}")
    #print(f"cities_not_matched: {[city for city in cities if city not in list(city_teams.keys())]}")
    return city_teams
    
def nhl_correlation():
    nhl_cities = cities.copy()
    #print(nhl_df.head())
    #print('--------Original Teams 2018 from nhl.csv------------')
    teams_2018 = nhl_df[nhl_df['year'] == 2018]
    #print(teams_2018)
    #print('--------Team Names------------')
    #team_names = teams_2018['team']
    #print(team_names)
    #print('--------Make Team as Index------')
    teams_2018 = teams_2018.set_index('team')
    #print(teams_2018)

    #print("cities_teams_mapping")
    city_teams = cities_nhl_teams_mapping(nhl_df, list(nhl_cities["Metropolitan area"]))
    #print(city_teams)
    
    #print("--------Cities set_index----------")
    nhl_cities.set_index("Metropolitan area", inplace=True)
    #print(nhl_cities)
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    #print("---------W/L by City--------")
    for city in list(city_teams.keys()):
        teams_in_city = city_teams[city]
        city_population = nhl_cities.loc[city]['Population (2016 est.)[8]']
        #print(city_population)
        wins = pd.to_numeric(teams_2018.loc[teams_in_city]['W']).sum()
        loss = pd.to_numeric(teams_2018.loc[teams_in_city]['L']).sum()
        total = wins + loss
        win_loss_ratio = wins/total
        win_loss_by_region.append(win_loss_ratio)
        population_by_region.append(pd.to_numeric(city_population))
        print("City:{} and teams_in_city:{} population:{} w/l:{}".format(city, teams_in_city, city_population, win_loss_ratio))
 
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    result = stats.pearsonr(population_by_region, win_loss_by_region)
    return result[0]


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [21]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from collections import defaultdict

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def cities_nba_teams_mapping(df, cities):
    teams_2018 = df[df['year'] == 2018]
    team_names = teams_2018['team']
    team_city_match_found = 0
    team_city_no_match = 0

    # Map of cleaned city name to original city name
    # outer split using the encoded en-dash '-' to b'\xe2\x80\x93'
    cleaned_cities = {bytes(city.split('City')[0].split('Area')[0].split(', D.C.')[0].strip(), 'utf-8').split(b'\xe2\x80\x93')[0].decode():city for city in cities}
    #print("No. of cities:{} and cleaned_cities:{}".format(len(cities), len(cleaned_cities)))
    #print(f"cities_teams_mapping :: Cleaned Cities {cleaned_cities}")
    
    city_teams = defaultdict(list)
    city_teams['Indianapolis'].append(b'Indiana Pacers*\xc2\xa0(5)'.decode('utf-8'))
    city_teams['San Francisco Bay Area'].append(b'Golden State Warriors*\xc2\xa0(2)'.decode('utf-8'))
    city_teams['Minneapolis–Saint Paul'].append(b'Minnesota Timberwolves*\xc2\xa0(8)'.decode('utf-8'))
    city_teams['Salt Lake City'].append(b'Utah Jazz*\xc2\xa0(5)'.decode('utf-8'))
    
    match_already_found = [team for team_list in city_teams.values() for team in team_list]
    print(f"match_already_found {match_already_found}")

    for team in team_names:
        if team in match_already_found:
            team_city_match_found += 1
        else:
            back_city_name = team.rsplit(' ', 1)[0].strip() # "New York Islanders" --> "New York"
            front_city_name = team.split(' ')[0].strip() # Toronto Maple Leafs --> Toronto
            if back_city_name in cleaned_cities:
                city_teams[cleaned_cities[back_city_name]].append(team)
                team_city_match_found += 1
            elif front_city_name in cleaned_cities:
                city_teams[cleaned_cities[front_city_name]].append(team)
                team_city_match_found += 1
            else:
                print(f"No city found for {team}")
                team_city_no_match += 1
    #print(f"cities_teams_mapping :: team_city_match_found={team_city_match_found}, team_city_no_match={team_city_no_match}")
    #print(f"cities_not_matched: {[city for city in cities if city not in list(city_teams.keys())]}")
    return city_teams

def nba_correlation():
    nba_cities = cities.copy()
    teams_2018 = nba_df[nba_df['year'] == 2018]
    teams_2018 = teams_2018.set_index('team')
    print(teams_2018)
    
    city_teams = cities_nba_teams_mapping(nba_df, list(nba_cities["Metropolitan area"]))
    print("\ncity_teams mapping:: {}".format(city_teams))
    nba_cities.set_index("Metropolitan area", inplace=True)
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    for city in list(city_teams.keys()):
        teams_in_city = city_teams[city]
        city_population = nba_cities.loc[city]['Population (2016 est.)[8]']
        wins = pd.to_numeric(teams_2018.loc[teams_in_city]['W']).sum()
        loss = pd.to_numeric(teams_2018.loc[teams_in_city]['L']).sum()
        total = wins + loss
        win_loss_ratio = wins/total
        win_loss_by_region.append(win_loss_ratio)
        population_by_region.append(pd.to_numeric(city_population))
        #print("City:{} teams_in_city:{} population:{} w/l:{}".format(city, teams_in_city, city_population, win_loss_ratio))
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    result = stats.pearsonr(population_by_region, win_loss_by_region)
    print(result)
    return result[0]



## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [22]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from collections import defaultdict

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def cities_mlb_teams_mapping(df, cities):
    teams_2018 = df[df['year'] == 2018]
    team_names = teams_2018['team']
    team_city_match_found = 0
    team_city_no_match = 0

    # Map of cleaned city name to original city name
    # outer split using the encoded en-dash '-' to b'\xe2\x80\x93'
    cleaned_cities = {bytes(city.split('City')[0].split('Area')[0].split(', D.C.')[0].strip(), 'utf-8').split(b'\xe2\x80\x93')[0].decode():city for city in cities}
    #print("No. of cities:{} and cleaned_cities:{}".format(len(cities), len(cleaned_cities)))
    #print(f"cities_teams_mapping :: Cleaned Cities {cleaned_cities}")
    
    city_teams = defaultdict(list)
    city_teams['San Francisco Bay Area'].append('Oakland Athletics')
    city_teams['San Francisco Bay Area'].append('San Francisco Giants')
    city_teams['Minneapolis–Saint Paul'].append('Minnesota Twins')
    city_teams['Phoenix'].append('Arizona Diamondbacks')
    city_teams['Dallas–Fort Worth'].append('Texas Rangers')
    city_teams['Denver'].append('Colorado Rockies')
    
    match_already_found = [team for team_list in city_teams.values() for team in team_list]
    print(f"match_already_found {match_already_found}")
    
    for team in team_names:
        if team in match_already_found:
            team_city_match_found += 1
        else:
            back_city_name = team.rsplit(' ', 1)[0].strip() # "New York Islanders" --> "New York"
            front_city_name = team.split(' ')[0].strip() # Toronto Maple Leafs --> Toronto
            if back_city_name in cleaned_cities:
                city_teams[cleaned_cities[back_city_name]].append(team)
                team_city_match_found += 1
            elif front_city_name in cleaned_cities:
                city_teams[cleaned_cities[front_city_name]].append(team)
                team_city_match_found += 1
            else:
                print(f"No city found for {team}")
                team_city_no_match += 1
    #print(f"cities_teams_mapping :: team_city_match_found={team_city_match_found}, team_city_no_match={team_city_no_match}")
    #print(f"cities_not_matched: {[city for city in cities if city not in list(city_teams.keys())]}")
    return city_teams
    
    
def mlb_correlation(): 
    mlb_cities = cities.copy()
    teams_2018 = mlb_df[mlb_df['year'] == 2018]
    teams_2018 = teams_2018.set_index('team')
    #print(teams_2018)
    
    city_teams = cities_mlb_teams_mapping(mlb_df, list(mlb_cities["Metropolitan area"]))
    print("\ncity_teams mapping:: {}".format(city_teams))
    mlb_cities.set_index("Metropolitan area", inplace=True)
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    for city in list(city_teams.keys()):
        teams_in_city = city_teams[city]
        city_population = mlb_cities.loc[city]['Population (2016 est.)[8]']
        wins = pd.to_numeric(teams_2018.loc[teams_in_city]['W']).sum()
        loss = pd.to_numeric(teams_2018.loc[teams_in_city]['L']).sum()
        total = wins + loss
        win_loss_ratio = wins/total
        win_loss_by_region.append(win_loss_ratio)
        population_by_region.append(pd.to_numeric(city_population))
        #print("City:{} teams_in_city:{} population:{} w/l:{}".format(city, teams_in_city, city_population, win_loss_ratio))
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    result = stats.pearsonr(population_by_region, win_loss_by_region)
    print(result)
    return result[0]


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from collections import defaultdict

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def cities_nfl_teams_mapping(df, cities):
    teams_2018 = df[df['year'] == 2018]
    team_names = teams_2018['team']
    team_city_match_found = 0
    team_city_no_match = 0

    # Map of cleaned city name to original city name
    # outer split using the encoded en-dash '-' to b'\xe2\x80\x93'
    cleaned_cities = {bytes(city.split('City')[0].split('Area')[0].split(', D.C.')[0].strip(), 'utf-8').split(b'\xe2\x80\x93')[0].decode():city for city in cities}
    #print("No. of cities:{} and cleaned_cities:{}".format(len(cities), len(cleaned_cities)))
    print(f"Cleaned Cities:: {cleaned_cities}")
    
    city_teams = defaultdict(list)
    city_teams['Boston'].append('New England Patriots*')
    city_teams['Nashville'].append('Tennessee Titans')
    city_teams['San Francisco Bay Area'].append('Oakland Raiders')
    city_teams['San Francisco Bay Area'].append('San Francisco 49ers')
    city_teams['Minneapolis–Saint Paul'].append('Minnesota Vikings')
    city_teams['Charlotte'].append('Carolina Panthers')
    city_teams['Phoenix'].append('Arizona Cardinals')
    
    match_already_found = [team for team_list in city_teams.values() for team in team_list]
    #print(f"match_already_found {match_already_found}")
    
    for team in team_names:
        if team in match_already_found:
            team_city_match_found += 1
        else:
            back_city_name = team.rsplit(' ', 1)[0].strip() # "New York Islanders" --> "New York"
            front_city_name = team.split(' ')[0].strip() # Toronto Maple Leafs --> Toronto
            if back_city_name in cleaned_cities:
                city_teams[cleaned_cities[back_city_name]].append(team)
                team_city_match_found += 1
            elif front_city_name in cleaned_cities:
                city_teams[cleaned_cities[front_city_name]].append(team)
                team_city_match_found += 1
            else:
                print(f"No city found for {team}")
                team_city_no_match += 1
    #print(f"cities_teams_mapping :: team_city_match_found={team_city_match_found}, team_city_no_match={team_city_no_match}")
    #print(f"cities_not_matched: {[city for city in cities if city not in list(city_teams.keys())]}")
    return city_teams
    
def nfl_correlation():
    nfl_cities = cities.copy()
    teams_2018 = nfl_df[nfl_df['year'] == 2018]
    teams_2018 = teams_2018.set_index('team')
    #print(teams_2018)

    city_teams = cities_nfl_teams_mapping(nfl_df, list(nfl_cities["Metropolitan area"]))
    print("\ncity_teams mapping:: {}".format(city_teams))
    nfl_cities.set_index("Metropolitan area", inplace=True)

    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    for city in list(city_teams.keys()):
        teams_in_city = city_teams[city]
        city_population = nfl_cities.loc[city]['Population (2016 est.)[8]']
        wins = pd.to_numeric(teams_2018.loc[teams_in_city]['W']).sum()
        loss = pd.to_numeric(teams_2018.loc[teams_in_city]['L']).sum()
        total = wins + loss
        win_loss_ratio = wins/total
        win_loss_by_region.append(win_loss_ratio)
        population_by_region.append(pd.to_numeric(city_population))
        #print("City:{} teams_in_city:{} population:{} w/l:{}".format(city, teams_in_city, city_population, win_loss_ratio))
     
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    result = stats.pearsonr(population_by_region, win_loss_by_region)
    print(result)
    return result[0]


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [24]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from collections import defaultdict

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def cities_nhl_team_mapping(df, cities):
    teams_2018 = df[df['year'] == 2018]
    team_names = teams_2018['team']
    team_city_match_found = 0
    team_city_no_match = 0
    
    # Map of cleaned city name to original city name
    cleaned_cities = {city.split('City')[0].split('Area')[0].split(', D.C.')[0].strip():city for city in cities}
    
    city_teams = defaultdict(list)
    city_teams['Phoenix'].append('Arizona Coyotes')
    city_teams['Las Vegas'].append('Vegas Golden Knights*')
    city_teams['Miami–Fort Lauderdale'].append('Florida Panthers')
    city_teams['Raleigh'].append('Carolina Hurricanes')
    city_teams['Denver'].append('Colorado Avalanche*')
    city_teams['Dallas–Fort Worth'].append('Dallas Stars')
    city_teams['San Francisco Bay Area'].append('San Jose Sharks*')
    city_teams['Minneapolis–Saint Paul'].append('Minnesota Wild*')
    city_teams['Los Angeles'].append('Anaheim Ducks*')
    city_teams['New York City'].append('New Jersey Devils*')
    
    match_already_found = [team for team_list in city_teams.values() for team in team_list]
    for team in team_names:
        #team = team.split('*')[0] # Some teams have '*' in the last
        if team in match_already_found:
            team_city_match_found += 1
        else:
            back_city_name = team.rsplit(' ', 1)[0].strip() # "New York Islanders" --> "New York"
            front_city_name = team.split(' ')[0].strip() # Toronto Maple Leafs --> Toronto
            if back_city_name in cleaned_cities:
                city_teams[cleaned_cities[back_city_name]].append(team)
                team_city_match_found += 1
            elif front_city_name in cleaned_cities:
                city_teams[cleaned_cities[front_city_name]].append(team)
                team_city_match_found += 1
            else:
                #print(f"No city found for {team}")
                team_city_no_match += 1
    #print(f"cities_teams_mapping :: team_city_match_found={team_city_match_found}, team_city_no_match={team_city_no_match}")
    #print(f"cities_not_matched: {[city for city in cities if city not in list(city_teams.keys())]}")
    return city_teams

def cities_nba_team_mapping(df, cities):
    teams_2018 = df[df['year'] == 2018]
    team_names = teams_2018['team']
    team_city_match_found = 0
    team_city_no_match = 0

    # Map of cleaned city name to original city name
    # outer split using the encoded en-dash '-' to b'\xe2\x80\x93'
    cleaned_cities = {bytes(city.split('City')[0].split('Area')[0].split(', D.C.')[0].strip(), 'utf-8').split(b'\xe2\x80\x93')[0].decode():city for city in cities}
    #print("No. of cities:{} and cleaned_cities:{}".format(len(cities), len(cleaned_cities)))
    #print(f"cities_teams_mapping :: Cleaned Cities {cleaned_cities}")
    
    city_teams = defaultdict(list)
    city_teams['Indianapolis'].append(b'Indiana Pacers*\xc2\xa0(5)'.decode('utf-8'))
    city_teams['San Francisco Bay Area'].append(b'Golden State Warriors*\xc2\xa0(2)'.decode('utf-8'))
    city_teams['Minneapolis–Saint Paul'].append(b'Minnesota Timberwolves*\xc2\xa0(8)'.decode('utf-8'))
    city_teams['Salt Lake City'].append(b'Utah Jazz*\xc2\xa0(5)'.decode('utf-8'))
    
    match_already_found = [team for team_list in city_teams.values() for team in team_list]
    for team in team_names:
        if team in match_already_found:
            team_city_match_found += 1
        else:
            back_city_name = team.rsplit(' ', 1)[0].strip() # "New York Islanders" --> "New York"
            front_city_name = team.split(' ')[0].strip() # Toronto Maple Leafs --> Toronto
            if back_city_name in cleaned_cities:
                city_teams[cleaned_cities[back_city_name]].append(team)
                team_city_match_found += 1
            elif front_city_name in cleaned_cities:
                city_teams[cleaned_cities[front_city_name]].append(team)
                team_city_match_found += 1
            else:
                #print(f"No city found for {team}")
                team_city_no_match += 1
    #print(f"cities_teams_mapping :: team_city_match_found={team_city_match_found}, team_city_no_match={team_city_no_match}")
    #print(f"cities_not_matched: {[city for city in cities if city not in list(city_teams.keys())]}")
    return city_teams

def cities_mlb_team_mapping(df, cities):
    teams_2018 = df[df['year'] == 2018]
    team_names = teams_2018['team']
    team_city_match_found = 0
    team_city_no_match = 0
    
    # Map of cleaned city name to original city name
    # outer split using the encoded en-dash '-' to b'\xe2\x80\x93'
    cleaned_cities = {bytes(city.split('City')[0].split('Area')[0].split(', D.C.')[0].strip(), 'utf-8').split(b'\xe2\x80\x93')[0].decode():city for city in cities}
    #print("No. of cities:{} and cleaned_cities:{}".format(len(cities), len(cleaned_cities)))
    #print(f"cities_teams_mapping :: Cleaned Cities {cleaned_cities}")
    
    city_teams = defaultdict(list)
    city_teams['San Francisco Bay Area'].append('Oakland Athletics')
    city_teams['San Francisco Bay Area'].append('San Francisco Giants')
    city_teams['Minneapolis–Saint Paul'].append('Minnesota Twins')
    city_teams['Phoenix'].append('Arizona Diamondbacks')
    city_teams['Dallas–Fort Worth'].append('Texas Rangers')
    city_teams['Denver'].append('Colorado Rockies')
    
    match_already_found = [team for team_list in city_teams.values() for team in team_list]
    for team in team_names:
        if team in match_already_found:
            team_city_match_found += 1
        else:
            back_city_name = team.rsplit(' ', 1)[0].strip() # "New York Islanders" --> "New York"
            front_city_name = team.split(' ')[0].strip() # Toronto Maple Leafs --> Toronto
            if back_city_name in cleaned_cities:
                city_teams[cleaned_cities[back_city_name]].append(team)
                team_city_match_found += 1
            elif front_city_name in cleaned_cities:
                city_teams[cleaned_cities[front_city_name]].append(team)
                team_city_match_found += 1
            else:
                #print(f"No city found for {team}")
                team_city_no_match += 1
    #print(f"cities_teams_mapping :: team_city_match_found={team_city_match_found}, team_city_no_match={team_city_no_match}")
    #print(f"cities_not_matched: {[city for city in cities if city not in list(city_teams.keys())]}")
    return city_teams

def cities_nfl_team_mapping(df, cities):
    teams_2018 = df[df['year'] == 2018]
    team_names = teams_2018['team']
    team_city_match_found = 0
    team_city_no_match = 0

    # Map of cleaned city name to original city name
    # outer split using the encoded en-dash '-' to b'\xe2\x80\x93'
    cleaned_cities = {bytes(city.split('City')[0].split('Area')[0].split(', D.C.')[0].strip(), 'utf-8').split(b'\xe2\x80\x93')[0].decode():city for city in cities}
    #print("No. of cities:{} and cleaned_cities:{}".format(len(cities), len(cleaned_cities)))
    #print(f"Cleaned Cities:: {cleaned_cities}")
    
    city_teams = defaultdict(list)
    city_teams['Boston'].append('New England Patriots*')
    city_teams['Nashville'].append('Tennessee Titans')
    city_teams['San Francisco Bay Area'].append('Oakland Raiders')
    city_teams['San Francisco Bay Area'].append('San Francisco 49ers')
    city_teams['Minneapolis–Saint Paul'].append('Minnesota Vikings')
    city_teams['Charlotte'].append('Carolina Panthers')
    city_teams['Phoenix'].append('Arizona Cardinals')
    
    match_already_found = [team for team_list in city_teams.values() for team in team_list]
    for team in team_names:
        if team in match_already_found:
            team_city_match_found += 1
        else:
            back_city_name = team.rsplit(' ', 1)[0].strip() # "New York Islanders" --> "New York"
            front_city_name = team.split(' ')[0].strip() # Toronto Maple Leafs --> Toronto
            if back_city_name in cleaned_cities:
                city_teams[cleaned_cities[back_city_name]].append(team)
                team_city_match_found += 1
            elif front_city_name in cleaned_cities:
                city_teams[cleaned_cities[front_city_name]].append(team)
                team_city_match_found += 1
            else:
                #print(f"No city found for {team}")
                team_city_no_match += 1
    #print(f"cities_teams_mapping :: team_city_match_found={team_city_match_found}, team_city_no_match={team_city_no_match}")
    #print(f"cities_not_matched: {[city for city in cities if city not in list(city_teams.keys())]}")
    return city_teams

def city_sports_performance(df, cities_team_mapping):
    cities_copy = cities.copy()
    teams_2018 = df[df['year'] == 2018]
    teams_2018 = teams_2018.set_index('team')
    #print(teams_2018)

    #print("cities_teams_mapping")
    city_teams = cities_team_mapping(df, list(cities_copy["Metropolitan area"]))
    #print(city_teams)
    
    # Cities Performance(W/L ratio) Series
    metropolitan_cities = pd.DataFrame(index=sorted(list(city_teams.keys())), columns=['Performance'])
    
    #print("---------W/L by City--------")
    for city in list(city_teams.keys()):
        teams_in_city = city_teams[city]
        wins = pd.to_numeric(teams_2018.loc[teams_in_city]['W']).sum()
        loss = pd.to_numeric(teams_2018.loc[teams_in_city]['L']).sum()
        total = wins + loss
        win_loss_ratio = wins/total
        metropolitan_cities.loc[city]['Performance'] = win_loss_ratio
    #print()
    #print(metropolitan_cities)
    return metropolitan_cities, city_teams

def sports_team_performance():
    nhl, nhl_city_teams = city_sports_performance(nhl_df, cities_nhl_team_mapping)
    print("NHL")
    print(nhl)
    nba, nba_city_teams = city_sports_performance(nba_df, cities_nba_team_mapping)
    print("\nNBA")
    print(nba)
    mlb, mlb_city_teams = city_sports_performance(mlb_df, cities_mlb_team_mapping)
    print("\nMLB")
    print(mlb)
    nfl, nfl_city_teams = city_sports_performance(nfl_df, cities_nfl_team_mapping)
    print("\nNFL")
    print(nfl)
    print()
    
    '''
    print(sorted([team for team_list in nhl_city_teams.values() for team in team_list]))
    print(sorted([team for team_list in nba_city_teams.values() for team in team_list]))
    print(sorted([team for team_list in mlb_city_teams.values() for team in team_list]))
    print(sorted([team for team_list in nfl_city_teams.values() for team in team_list]))
    print("NHL NBA Common_teams: {}".format(set([team for team_list in nhl_city_teams.values() for team in team_list]) & 
                                     set([team for team_list in nba_city_teams.values() for team in team_list])))
    print("NHL MLB Common_teams: {}".format(set([team for team_list in nhl_city_teams.values() for team in team_list]) & 
                                     set([team for team_list in mlb_city_teams.values() for team in team_list])))
    print("NHL NFL Common_teams: {}".format(set([team for team_list in nhl_city_teams.values() for team in team_list]) & 
                                     set([team for team_list in nfl_city_teams.values() for team in team_list])))
    print("NBA MLB Common_teams: {}".format(set([team for team_list in nba_city_teams.values() for team in team_list]) & 
                                     set([team for team_list in mlb_city_teams.values() for team in team_list])))
    print("NBA NFL Common_teams: {}".format(set([team for team_list in nba_city_teams.values() for team in team_list]) & 
                                     set([team for team_list in nfl_city_teams.values() for team in team_list])))
    print("MLB NFL Common_teams: {}".format(set([team for team_list in mlb_city_teams.values() for team in team_list]) & 
                                     set([team for team_list in nfl_city_teams.values() for team in team_list])))
    print("Common_teams: {}".format(set([team for team_list in nhl_city_teams.values() for team in team_list]) & 
                                     set([team for team_list in nba_city_teams.values() for team in team_list]) &
                                     set([team for team_list in mlb_city_teams.values() for team in team_list]) &
                                     set([team for team_list in nfl_city_teams.values() for team in team_list])))
    '''
    
    print("NHL NBA")
    nhl_nba_merge = pd.merge(nhl, nba, left_index=True, right_index=True, suffixes=('_nhl', '_nba'), indicator=True)
    print(nhl_nba_merge)
    nhl_nba_p_values = stats.ttest_rel(nhl_nba_merge['Performance_nhl'], nhl_nba_merge['Performance_nba'])
    print("\nnhl_nba_p_values {}\n".format(nhl_nba_p_values))
    
    print("NHL MLB")
    nhl_mlb_merge = pd.merge(nhl, mlb, left_index=True, right_index=True, suffixes=('_nhl', '_mlb'), indicator=True)
    print(nhl_mlb_merge)
    nhl_mlb_p_values = stats.ttest_rel(nhl_mlb_merge['Performance_nhl'], nhl_mlb_merge['Performance_mlb'])
    print("\nhl_mlb_p_values {}\n".format(nhl_mlb_p_values))
    
    print("NHL NFL")
    nhl_nfl_merge = pd.merge(nhl, nfl, left_index=True, right_index=True, suffixes=('_nhl', '_nfl'), indicator=True)
    print(nhl_nfl_merge)
    nhl_nfl_p_values = stats.ttest_rel(nhl_nfl_merge['Performance_nhl'], nhl_nfl_merge['Performance_nfl'])
    print("\nnhl_nfl_p_values {}\n".format(nhl_nfl_p_values))
    
    print("NBA MLB")
    nba_mlb_merge = pd.merge(nba, mlb, left_index=True, right_index=True, suffixes=('_nba', '_mlb'), indicator=True)
    print(nba_mlb_merge)
    nba_mlb_p_values = stats.ttest_rel(nba_mlb_merge['Performance_nba'], nba_mlb_merge['Performance_mlb'])
    print("\nba_mlb_p_values {}\n".format(nba_mlb_p_values))
    
    print("NBA NFL")
    nba_nfl_merge = pd.merge(nba, nfl, left_index=True, right_index=True, suffixes=('_nba', '_nfl'), indicator=True)
    print(nba_nfl_merge)
    nba_nfl_p_values = stats.ttest_rel(nba_nfl_merge['Performance_nba'], nba_nfl_merge['Performance_nfl'])
    print("\nnba_nfl_p_values {}\n".format(nba_nfl_p_values))
    
    print("MLB NFL")
    mlb_nfl_merge = pd.merge(mlb, nfl, left_index=True, right_index=True, suffixes=('_mlb', '_nfl'), indicator=True)
    print(mlb_nfl_merge)
    mlb_nfl_p_values = stats.ttest_rel(mlb_nfl_merge['Performance_mlb'], mlb_nfl_merge['Performance_nfl'])
    print("\nmlb_nfl_p_values {}\n".format(mlb_nfl_p_values))
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    p_values.loc['NHL']['NBA'] = p_values.loc['NBA']['NHL'] = nhl_nba_p_values[1]
    p_values.loc['NHL']['MLB'] = p_values.loc['MLB']['NHL'] = nhl_mlb_p_values[1]
    p_values.loc['NHL']['NFL'] = p_values.loc['NFL']['NHL'] = nhl_nfl_p_values[1]
    p_values.loc['NBA']['MLB'] = p_values.loc['MLB']['NBA'] = nba_mlb_p_values[1]
    p_values.loc['NBA']['NFL'] = p_values.loc['NFL']['NBA'] = nba_nfl_p_values[1]
    p_values.loc['MLB']['NFL'] = p_values.loc['NFL']['MLB'] = mlb_nfl_p_values[1]
    print(p_values)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
